The tutorial that is followed in this notebook is found [here](https://www.tensorflow.org/versions/r0.11/tutorials/recurrent/index.html).

In [6]:
import tensorflow as tf
import numpy as np
from tensorflow.models.rnn.ptb import reader

flags = tf.flags
flags.DEFINE_string("data_path", "./simple-examples/data",
                   "Where the training/test data is stored.")
flags.DEFINE_string("save_path", "./model",
                   "Model output directory.")
flags.DEFINE_bool("use_fp16", False,
                 "Train using 16-bit floats instead of 32bit floats.")

FLAGS = flags.FLAGS

In [7]:
def data_type():
    return tf.float16 if FLAGS.use_fp16 else tf.float32

class PTBInput(object):
  """The input data."""
  def __init__(self, config, data, name=None):
    self.batch_size = batch_size = config.batch_size
    self.num_steps = num_steps = config.num_steps
    self.epoch_size = ((len(data) // batch_size) - 1) // num_steps
    self.input_data, self.targets = reader.ptb_producer(
        data, batch_size, num_steps, name=name)

In [8]:
class PTBModel(object):
    """The PTB model."""
    def __init__(self, is_training, config, input_):
        self._input = input_
        
        batch_size = input_.batch_size
        num_steps = input_.num_steps
        size = config.hidden_size
        vocab_size = config.vocab_size
        
        # Here is the implementation of the LSTM cells
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(
            size, 
            forget_bias=0.0, 
            state_is_tuple=True)